In [1]:
import os
try:
    from google.colab import drive
    COLAB = True
    print("Estoy en Google CoLab")
    %tensorflow_version 2.x
    !pip install livelossplot
except:
    os.environ["CUDA_VISIBLE_DEVICES"]="0" 
    print("No estoy en Google CoLab")
    COLAB = False

Estoy en Google CoLab
     |████████████████████████████████| 793 kB 19.8 MB/s 
     |████████████████████████████████| 381 kB 43.9 MB/s 
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.29 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.33.0 which is incompatible.


In [2]:
from urllib import request
from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.utils import np_utils
from keras.layers import Dense, LSTM, SimpleRNN, GRU, Input, Embedding, Dropout, Bidirectional
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.optimizers import RMSprop
from keras import layers
try:
    from livelossplot import PlotLossesKerasTF
except:
    !pip install livelossplot
    from livelossplot import PlotLossesKerasTF
    
import tensorflow as tf 
import re
import os

ImportError: ignored

In [ ]:
def train(model, train_data, optimizer, epochs = 30, loss='mse', batch_size=32, plot=True):
    model.compile(
      optimizer=optimizer,
      loss=loss,
      metrics=['accuracy'],
    )
    callbacks = None
    if plot:
        callbacks = [PlotLossesKerasTF()]
    
    x_train, y_train = train_data
    model.fit(
      x_train, 
      y_train,
      epochs=epochs,
      batch_size=batch_size,
      callbacks=callbacks,
      shuffle = True
    )
    
    return model

## Generación de textos

En este ejemplo crearemos un generador de textos. Para ello vamos a usar como corpus "Don Quijote" ya que tiene un lenguaje peculiar para nuestra época y nos resultará curioso ver que aprende nuestro modelo.

### El corpus

Podemos descargar el texto del proyecto Gutemberg:

In [ ]:
import requests
url = 'http://www.gutenberg.org/cache/epub/2000/pg2000.txt'
corpus = requests.get(url).content.decode("utf-8").split("\r\n")

In [ ]:
corpus[:10000]

Lo primero que debemos hacer es limpiar el texto. En nuestro caso vamos a convertir a minúsculas todo, quitaremos los retornos de carro y los signos de puntuación:

In [ ]:
seq_length = 20
start_story = '| ' * seq_length

text = corpus[605:-370]    # eliminamos la primeras y últimas líneas que no corresponde al libro
text = ' '.join(text) # unificamos en una sola cadena de texto
text = text.replace('\r\n\r\n\r\n\r\n\r\n\r\n', start_story)
text = text.lower() # convertimos a minúscula
text = text.replace('\r\n', ' ') # eliminamos los retorno de carro
#text = re.sub('  +', '. ', text).strip() # sustituimos dos espacios o más consecutivos por un "."
text = text.replace('..', '.')
text = re.sub('([¡!"#$%&()*+,-./:;<=>¿?@[\]^_`{|}~])', r' \1 ', text) # separamos esos símbolos del texto
text = re.sub('\s{2,}', ' ', text)

In [ ]:
text[:1000]

El siguiente paso es tokenizar, es decir particionar el texto (lo podemos particionar por token o por carácter):

In [ ]:
tokenizer = Tokenizer(char_level = False, filters = '')
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
token_list = tokenizer.texts_to_sequences([text])[0]

Podemos mostrar una lista de nuestro diccionario:

In [ ]:
[(tokenizer.index_word[i], i) for i in token_list[:10]]

In [ ]:
'Total de palabras en el vocabulario: ', len(tokenizer.index_word.keys())

### El conjunto de entrenamiento

La red se entrenará para predecir el siguiente token de la secuencia dada una sencuencia previa de tokens. Por ejemplo, si le damos al modelo los tokens *era de complexión*, se podría esperar que dinera como predicción el token *recia* o *fuerte*.

La longitud de entrada (lo tokens previos al token a predecir) es un parámetro del optimizar. En este caso usamores una longitud de 20, por lo que debemos dividir el texto en trozos de 20 palabras. Esto nos dará un conjunto de datos de 444145 (ya que el corpus completo tiene 444165 tokens).

In [ ]:
len(token_list)

In [ ]:
def generate_sequences(token_list, step):
    X, y = [], []
    
    for i in range(0, len(token_list) - seq_length, step):
        X.append(token_list[i: i + seq_length])
        y.append(token_list[i + seq_length])
        
    print('Total de secuencias:', len(X))
    
    return np.array(X), np.array(y)

X, y = generate_sequences(token_list, 1)

### Primera red recurrente

Nuestra arquitectura será una red recurrente que usará embeddings antes de las células recurrentes. En este caso dichos embeddings los tendrá que aprender la red durante el entrenamiento:

In [ ]:
epochs = 20
batch_size = 256
embedding_size = 100
n_units = 64

text_in = Input(shape = (None,))
x = text_in
x = Embedding(total_words, embedding_size)(x)
x = GRU(n_units, )(x)
x = Dropout(0.2)(x)
text_out = Dense(total_words, activation = 'softmax')(x)

gru = Model(text_in, text_out)
gru.summary()              

In [ ]:
train(gru, (X, y), optimizer='adam', loss='sparse_categorical_crossentropy', 
      batch_size=batch_size, epochs=epochs)



Vamos a hacer un generador de texto y vamos a unificar las partes comunes en un clase que nos ayuda a entrenar este tipo de modelos solo cambiando el model:

In [ ]:
class Writer:
    
    def __init__(self, next_token_model, tokenizer=None, mode='word', start_token='|', end_token='.'):
        self.next_token_model = next_token_model
        self.tokenizer = tokenizer
        self.mode = mode
        self.start_token='|'
        self.end_token='.'

        
    def __generate_sequences(self, token_list, step=1, seq_length=20):
        X, y = [], []
    
        for i in range(0, len(token_list) - seq_length, step):
            X.append(token_list[i: i + seq_length])
            y.append(token_list[i + seq_length])
        
        print('Total de secuencias:', len(X))
    
        return np.array(X), np.array(y)
    
    def __train(self, train_data, optimizer='adam', epochs = 30, batch_size=32):
        loss = 'sparse_categorical_crossentropy'
        self.next_token_model.compile(
          optimizer=optimizer,
          loss=loss,
          metrics=['accuracy'],
        )

        x_train, y_train = train_data
        self.next_token_model.fit(
          x_train, 
          y_train,
          epochs=epochs,
          batch_size=batch_size,
          callbacks=[PlotLossesKeras(), EarlyStopping(patience=5,
                                                      restore_best_weights=True)],
          shuffle = True
        )


    def train(self, corpus, optimizer='adam', epochs = 30, batch_size=32, seq_length=20):
        self.tokenizer.fit_on_texts([text])
        total_words = len(self.tokenizer.word_index) + 1
        token_list = tokenizer.texts_to_sequences([text])[0]
        
        x_train, y_train = self.__generate_sequences(token_list, seq_length=seq_length)
        
        self.__train((x_train, y_train), optimizer=optimizer, 
                     epochs=epochs, batch_size=batch_size)

    
    def __sample(self, preds, temperature=1.0):
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / (np.sum(exp_preds)+ 1e-5)
        probas = np.random.multinomial(1, preds, 1)
        return np.argmax(probas)
    
    def write(self, start_tokens, temperature=1.0, verbose=True, max_len=10000):
        sentence = start_tokens
        if verbose:
            print(sentence, end='')
            

        while len(sentence) < max_len:
            context = np.array(tokenizer.texts_to_sequences([sentence]))
            prob = self.next_token_model([context])
            
            index = self.__sample(prob[0,:], temperature)
            next_token = tokenizer.index_word[index]

            sep = '' if self.mode=='char' else ' '
            if verbose:
                print(sep + next_token, end='')
            
            
            sentence += sep + next_token
            
            if next_token == self.end_token:
                break


        return sentence

In [ ]:
tf.get_logger().setLevel('INFO')
writer = Writer(gru, tokenizer)
results = writer.write('el perro estaba hambriento', temperature=1)

In [ ]:
results = writer.write('el perro estaba hambriento', temperature=3)

In [ ]:
results = writer.write('el perro estaba hambriento', temperature=0.1)

In [ ]:
gru.save('gru_quijote.h5')